In [ ]:
!apt-get update

!pip install numpy
!pip install scipy
!pip install pandas
!pip install h5py
!pip install sys
!pip install os
!pip install time
!pip install glob
!pip install matplotlib
!pip install seaborn
!pip install Ipython
!pip install warnings
!pip install keras
!pip install tensorflow

In [1]:
# Import several packages that will be used throughout

# numeric packages
import numpy as np
import scipy
import scipy.io
import pandas as pd
import h5py

# filesystem and OS
import sys, os, time
import glob

# plotting
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
from IPython.display import clear_output


import warnings
warnings.filterwarnings('ignore')

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

/media/mixalis/Seagate Expansion Drive/TU Delft/DeepLearning/Project/.env/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
#from imagenet_utils import preprocess_input, decode_predictions
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from tensorflow.contrib.metrics import streaming_mean_iou
from keras.objectives import *
import keras.backend as K

Using TensorFlow backend.


In [3]:
sys.path.append("../classifier/keras-utils/")
sys.path.append("../classifier/keras-models/")

# from multi_gpu import make_parallel
import keras_utils as ku
# from vgg16 import vgg16

In [5]:
# workdir = "/home/adalbert/nbserver/tf-workspace/deepsat-experiments/"
# os.chdir(workdir)

# Import data and set up batching 

### Set up batching

In [4]:
BATCH_SIZE = 50

train_dir = "../deepsat-data/img/train/"
test_dir = "../deepsat-data/img/test/"

In [5]:
# Generator for preprocessing images for data augmentation

from keras.preprocessing.image import ImageDataGenerator

def apply_mean(image_data_generator):
    """Subtracts the VGG dataset mean"""
    image_data_generator.mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3, 1, 1))

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=[1,1.2],
        vertical_flip=True,
        rotation_range=15,
        horizontal_flip=True)
apply_mean(train_datagen)


# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)
apply_mean(test_datagen)

train_generator = train_datagen.flow_from_directory(train_dir, \
                                         batch_size=BATCH_SIZE,
                                         target_size=(224,224),
                                         shuffle=True)
    
# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(test_dir, \
                                         batch_size=BATCH_SIZE,
                                         target_size=(224,224),
                                         shuffle=True)

# get class labels
class2ind = train_generator.class_indices
ind2class = {v:k for k,v in class2ind.items()}

N_CLASSES = len(ind2class)

Found 324000 images belonging to 6 classes.
Found 81000 images belonging to 6 classes.


In [6]:
for Xbatch, ybatch in train_generator:
    print(Xbatch.shape, ybatch.shape, Xbatch.min(), Xbatch.max())
    break
    
x_train = np.zeros(shape=(len(train_generator)/200,672,672,3),dtype=np.float32)
y_train = np.zeros(shape=(len(train_generator)/200,672,672,1),dtype=np.float32)


x_test = np.zeros(shape=(len(test_generator)/200,672,672,3),dtype=np.float32)
y_test = np.zeros(shape=(len(test_generator)/200,672,672,1),dtype=np.float32)

for i in range(len(train_generator)/200):
    temp = train_generator.next()
    for k in range(672): 
        for l in range(3):
            x_train[i][k][l*224:(l+1)*224] = temp[0][(k/224)*3+l][k%224]
            y_train[i][k][l*224:(l+1)*224] = np.argmax(temp[1][(k/224)*3+l])
  
for i in range(len(test_generator)/200):
    temp = test_generator.next()
    for k in range(672): 
        for l in range(3):
            x_test[i][k][l*224:(l+1)*224] = temp[0][(k/224)*3+l][k%224]
            y_test[i][k][l*224:(l+1)*224] = np.argmax(temp[1][(k/224)*3+l])



train_datagen.fit(x_train)
test_datagen.fit(x_test)

((50, 224, 224, 3), (50, 6), 0.0, 0.98304904)


# Define architectures to use
Load weights from model pretrained on ImageNet

#### VGG-16, pre-trained on ImageNet
Note that the weights are trained on BGR data for this architecture (probably following Caffe's convention). It turns out that they're a good enough starting point even if using RGB images.

In [ ]:
# build the convolutional base of the VGG16 network
model = vgg16(n_classes=N_CLASSES, input_shape=(224,224,3), fcn=False)

weights_file = "../vgg16_weights.h5"
model = ku.load_weights_into_model(model, weights_file, transpose_conv=True, 
                        layers_to_skip=["dense8"])

freeze_layers = [] #['conv1', 'conv2']
for l in model.layers:
    l.trainable = len([x for x in freeze_layers if x in l.name])==0 and \
                    len(l.get_weights())>0

for l in model.layers:
    print l.name, [x.sum() for x in l.get_weights()], l.trainable

#### ResNet initialized with ImageNet weights

In [7]:
sys.path.append("../deep-learning-models/")
sys.path.append("../classifier/")

from keras.layers import Flatten, Dense, Convolution2D
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
from FCNresnet50 import ResNet50
from FCNresnet50 import BilinearUpSampling2D
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions

# Softmax cross-entropy loss function for pascal voc segmentation
# and models which do not perform softmax.
# tensorlow only
def softmax_sparse_crossentropy_ignoring_last_label(y_true, y_pred):
    y_pred = K.reshape(y_pred, (-1, K.int_shape(y_pred)[-1]))
    log_softmax = tf.nn.log_softmax(y_pred)

    y_true = K.one_hot(tf.to_int32(K.flatten(y_true)), K.int_shape(y_pred)[-1]+1)
    unpacked = tf.unstack(y_true, axis=-1)
    y_true = tf.stack(unpacked[:-1], axis=-1)

    cross_entropy = -K.sum(y_true * log_softmax, axis=1)
    cross_entropy_mean = K.mean(cross_entropy)

    return cross_entropy_mean



def sparse_accuracy_ignoring_last_label(y_true, y_pred):
    nb_classes = K.int_shape(y_pred)[-1]
    y_pred = K.reshape(y_pred, (-1, nb_classes))
    y_true = K.one_hot(tf.to_int32(K.flatten(y_true)),
                       nb_classes + 1)
    unpacked = tf.unstack(y_true, axis=-1)
    legal_labels = ~tf.cast(unpacked[-1], tf.bool)
    y_true = tf.stack(unpacked[:-1], axis=-1)

    return K.sum(tf.to_float(legal_labels & K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))) / K.sum(tf.to_float(legal_labels))



model = ResNet50(weights=None, include_top=True,input_shape=(672,672,3),classes=6)

#model.layers.pop() # Get rid of the classification layer
# model.layers.pop()
# model.layers.pop()
#model.outputs = [model.layers[-1].output]
#model.output_layers = [model.layers[-1]] # added this line in addition to zo7 solution
#model.layers[-1].outbound_nodes = []

# fcn = Convolution2D(N_CLASSES, (1, 1), kernel_initializer='he_normal', activation='linear', 
#                       padding='valid', strides=(3, 3))(model.layers[-1].output)
# fcn = BilinearUpSampling2D(size=(32, 32))(fcn)
# model = Model(input=model.input, output=fcn)

#newClassificationLayer = Dense(N_CLASSES, activation='softmax')(model.layers[-1].output)
#model = Model(input=model.input, output=newClassificationLayer)

#### Load previously-saved model

In [17]:
# # load model from checkpoint
# model_file = "vgg16-deepsat-best-checkpoint.h5"

# # with tf.device('/cpu:0'):
# # load model and weights
# model = keras.models.load_model(model_file)

Spread model computation on multiple GPUs

In [12]:
GPUS = [0,1,2,3]

# this uses the TensorFlow backend to spread computation on multiple GPUs
model_gpu = make_parallel(model, GPUS)

NameError: name 'make_parallel' is not defined

In [ ]:
tf.__version__, keras.__version__

# Fine-tune model

Define model behavior with callbacks and compile

In [8]:
from keras.callbacks import ModelCheckpoint

# callback for a custom learning rate decay schedule

LR_DECAY_PER_EPOCH = 2.0 #1.1
BASE_LR = 1

lr_scheduler = lambda epoch: BASE_LR * LR_DECAY_PER_EPOCH**(-(epoch/10))
lr_decay_callback = keras.callbacks.LearningRateScheduler(lr_scheduler)

# callback to checkpoint best model
model_checkpoint_callback = ModelCheckpoint("resnet-deepsat-imagenet-best-checkpoint.h5", monitor='val_acc', \
                                      verbose=1, save_best_only=True, mode='max')

# compile model
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
model.compile(loss=softmax_sparse_crossentropy_ignoring_last_label, \
            metrics=[sparse_accuracy_ignoring_last_label],\
            optimizer=Adadelta(lr=BASE_LR))

Compile and train model

In [ ]:
# Logs to TensorBoard, new one for each run

log_path_tensorboard = "./logs/"

from keras.callbacks import TensorBoard

# now = time.strftime("%c")
now = str(int(time.time()))
tensorboard_callback_fn = TensorBoard(log_dir=log_path_tensorboard + now, \
                                histogram_freq=1, \
                                write_graph=True, \
                                write_images=False)

# Train model

history = model.fit_generator(
                train_datagen.flow(x_train, y_train, batch_size=2),
                samples_per_epoch=2000,
                nb_epoch=50,
                validation_data=test_datagen.flow(x_test, y_test,batch_size=2),
                callbacks = [lr_decay_callback, \
                            model_checkpoint_callback],
                nb_val_samples=1000)


Epoch 1/50
   2/1000 [..............................] - ETA: 5:33:27 - loss: 6.1793 - sparse_accuracy_ignoring_last_label: 0.1280

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))
ax[0].plot(history.history['loss'], label="train")
ax[0].plot(history.history['val_loss'], label="test")
ax[0].set_title("Loss", fontsize=14)
ax[0].set_xlabel("Epoch")
ax[0].legend(loc="best")
ax[1].plot(history.history['acc'], label="train")
ax[1].plot(history.history['val_acc'], label="test")
ax[1].set_title("Accuracy", fontsize=14)
ax[1].set_xlabel("Epoch")
ax[1].legend(loc="best")
plt.show()

In [ ]:
tf.__version__